In [1]:
!pip -q install --upgrade pip
!pip -q install "transformers>=4.41,<5" "torch>=2.1" "datasets>=2.20" fastapi uvicorn pydantic==2.* --progress-bar off


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.7 MB/s eta 0:00:00


In [3]:
!pip install torchtext --quiet


In [5]:
from datasets import load_dataset

# Load IMDB dataset
dataset = load_dataset("imdb")

# Preview first 5 examples
for i in range(5):
    print(f"Label: {dataset['train'][i]['label']} Review: {dataset['train'][i]['text'][:200]}...\n")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Label: 0 Review: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ev...

Label: 0 Review: "I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that ...

Label: 0 Review: If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches ...

Label: 0 Review: This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realistic acting (2) the impre...

Label: 0 Review: Oh, brother...after hearing about this ridiculous film for umpteen years all I can think of is 

In [6]:
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Test on a sample review
sample_review = dataset['train'][0]['text']
result = sentiment_model(sample_review[:512])[0]  # truncate to first 512 chars
print(result)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


{'label': 'NEGATIVE', 'score': 0.9913416504859924}


In [7]:
# Function to predict sentiment for multiple reviews
def predict_sentiments(reviews, model, max_samples=10):
    for i, review in enumerate(reviews[:max_samples]):
        result = model(review[:512])[0]  # truncate to 512 chars
        print(f"Review {i+1}:")
        print(f"Text: {review[:200]}...")  # show first 200 chars
        print(f"Predicted Sentiment: {result['label']}, Confidence: {result['score']:.2f}\n")

# Run on first 10 training reviews
sample_reviews = [dataset['train'][i]['text'] for i in range(10)]
predict_sentiments(sample_reviews, sentiment_model)


Review 1:
Text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ev...
Predicted Sentiment: NEGATIVE, Confidence: 0.99

Review 2:
Text: "I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that ...
Predicted Sentiment: NEGATIVE, Confidence: 1.00

Review 3:
Text: If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches ...
Predicted Sentiment: NEGATIVE, Confidence: 1.00

Review 4:
Text: This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realist

In [8]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
import uvicorn

# Allow FastAPI to run inside Colab
nest_asyncio.apply()

# Define input schema
class Review(BaseModel):
    text: str

# Initialize app
app = FastAPI(title="Sentiment Analysis API")

# API endpoint
@app.post("/predict")
def predict_sentiment(review: Review):
    try:
        result = sentiment_model(review.text[:512])[0]  # truncate input
        return {"label": result['label'], "score": result['score']}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# Run app
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [150]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [150]


In [11]:
import requests

# Colab local host URL
url = "http://127.0.0.1:8000/predict"

# Test review
test_review = {"text": "I absolutely loved this movie! The acting was fantastic."}

response = requests.post(url, json=test_review)
print(response.json())


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79c74e646ff0>: Failed to establish a new connection: [Errno 111] Connection refused'))